In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
data = load_iris()
X_train, X_test, Y_train, Y_test = train_test_split(data.data, data.target, test_size=0.4, random_state=42)

In [ ]:
X_val,X_test,Y_val,Y_test = train_test_split(X_test,Y_test,test_size=0.5,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from re import X
X_train = torch.tensor(X_train,dtype=torch.float32)
X_val = torch.tensor(X_val,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
Y_train = torch.tensor(Y_train)
Y_val = torch.tensor(Y_val)
Y_test = torch.tensor(Y_test)
X_train = X_train.to(device)
X_val = X_val.to(device)
X_test = X_test.to(device)
Y_train = Y_train.to(device)
Y_val = Y_val.to(device)
Y_test = Y_test.to(device)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(4,8),
    torch.nn.Linear(8,8),
    torch.nn.ReLU(),
    torch.nn.Linear(8,3)
)
model = model.to(device)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(model.parameters(),lr=0.01)

In [ ]:
def evaluate(model, X_val,Y_val):
  model.eval()
  with torch.no_grad():
    Y_pred = model(X_val)
  Y_pred = torch.argmax(Y_pred,dim=1)
  return sum(Y_pred == Y_val)/len(Y_val)

In [ ]:
max_epoch = 20
losses = []
for epoch in range(max_epoch):
  epoch_loss = []
  for x_train,y_train in zip(X_train,Y_train):
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss_value = loss(y_pred,y_train)
    epoch_loss.append(loss_value.item())
    loss_value.backward()
    optimizer.step()
  avg_loss = sum(epoch_loss)/len(epoch_loss)
  losses.append(avg_loss)
  acc = evaluate(model,X_val,Y_val)
  print(f"Epoch[{epoch+1}/{max_epoch}], avg_loss : {avg_loss}, Accuracy_val: {acc}")


Epoch[1/20], avg_loss : 0.6465594552353853, Accuracy_val: 0.8999999761581421
Epoch[2/20], avg_loss : 0.2770609828276469, Accuracy_val: 0.9666666388511658
Epoch[3/20], avg_loss : 0.15187056716151548, Accuracy_val: 1.0
Epoch[4/20], avg_loss : 0.11506682384044527, Accuracy_val: 1.0
Epoch[5/20], avg_loss : 0.10090616687210867, Accuracy_val: 1.0
Epoch[6/20], avg_loss : 0.09317987881233118, Accuracy_val: 1.0
Epoch[7/20], avg_loss : 0.08694786684076866, Accuracy_val: 1.0
Epoch[8/20], avg_loss : 0.08201448643711286, Accuracy_val: 1.0
Epoch[9/20], avg_loss : 0.07867887225188844, Accuracy_val: 1.0
Epoch[10/20], avg_loss : 0.07571352213532323, Accuracy_val: 1.0
Epoch[11/20], avg_loss : 0.07294230908300889, Accuracy_val: 1.0
Epoch[12/20], avg_loss : 0.07114405926018187, Accuracy_val: 1.0
Epoch[13/20], avg_loss : 0.07029419586354327, Accuracy_val: 1.0
Epoch[14/20], avg_loss : 0.07023936839591877, Accuracy_val: 1.0
Epoch[15/20], avg_loss : 0.07135033673536219, Accuracy_val: 1.0
Epoch[16/20], avg_los

In [ ]:
model.eval()
with torch.no_grad():
  Y_pred = model(X_test)
Y_pred = torch.argmax(Y_pred,dim=1)
print(sum(Y_pred == Y_test)/len(Y_test))

tensor(0.9667)
